# Assignment 2

### Python Version: 3.6
* conda create --name py3.6 python=3.6
* source activate py3.6
* python -V

### Java Version:  8
* brew cask install adoptopenjdk/openjdk/adoptopenjdk8
* open ~/.bash_profile
* export JAVA_HOME=$(/usr/libexec/java_home -v 1.8)
* java -version

## Step 01: Import Spark Session and initialize Spark

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local").appName("Weather Forecast").getOrCreate()

## Step 02: Load the dataset and print the schema and total number of entries

In [3]:
df = spark.read.csv('weatherAUS.csv', header=True)

In [4]:
df.describe().collect()

[Row(summary='count', Date='142193', Location='142193', MinTemp='142193', MaxTemp='142193', Rainfall='142193', Evaporation='142193', Sunshine='142193', WindGustDir='142193', WindGustSpeed='142193', WindDir9am='142193', WindDir3pm='142193', WindSpeed9am='142193', WindSpeed3pm='142193', Humidity9am='142193', Humidity3pm='142193', Pressure9am='142193', Pressure3pm='142193', Cloud9am='142193', Cloud3pm='142193', Temp9am='142193', Temp3pm='142193', RainToday='142193', RainTomorrow='142193'),
 Row(summary='mean', Date=None, Location=None, MinTemp='12.186399728729098', MaxTemp='23.226784191272444', Rainfall='2.3499740743111954', Evaporation='5.469824216349123', Sunshine='7.624853113193571', WindGustDir=None, WindGustSpeed='39.98429165757619', WindDir9am=None, WindDir3pm=None, WindSpeed9am='14.001988000994', WindSpeed3pm='18.63757586179718', Humidity9am='68.8438103105705', Humidity3pm='51.482606091656265', Pressure9am='1017.6537584159781', Pressure3pm='1015.258203537907', Cloud9am='4.437189391

In [5]:
df = spark.read.csv('weatherAUS.csv', header=True)

In [6]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: string (nullable = true)
 |-- MaxTemp: string (nullable = true)
 |-- Rainfall: string (nullable = true)
 |-- Evaporation: string (nullable = true)
 |-- Sunshine: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: string (nullable = true)
 |-- Humidity9am: string (nullable = true)
 |-- Humidity3pm: string (nullable = true)
 |-- Pressure9am: string (nullable = true)
 |-- Pressure3pm: string (nullable = true)
 |-- Cloud9am: string (nullable = true)
 |-- Cloud3pm: string (nullable = true)
 |-- Temp9am: string (nullable = true)
 |-- Temp3pm: string (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)



In [7]:
df.count()

142193

## Step 03: Delete columns from the dataset

In [8]:
df = df.drop('Date', 'Location', 'Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm')
df.show(0)

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
only showing top 0 rows



## Step 04: Print the number of missing data in each column.

In [9]:
from pyspark.sql.functions import col, sum

df.select(*(sum(col(c).isin('NA').cast("int")).alias(c) for c in df.columns)).show()

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|    637|    322|    1406|       9330|         9270|     10013|      3778|        1348|        2630|       1774|       3610|      14014|      13981|     1406|           0|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+



## Step 05: Fill the missing data with average value and maximum occurrence value.

In [10]:
from pyspark.sql.functions import avg

df.select(avg('MinTemp'),
          avg('MaxTemp'),
          avg('Rainfall'),
          avg('WindGustSpeed'),
          avg('WindSpeed9am'),
          avg('WindSpeed3pm'),
          avg('Humidity9am'),
          avg('Humidity3pm'),
          avg('Pressure9am'),
          avg('Pressure3pm')).show()

avg_list = df.select(avg('MinTemp'), 
                     avg('MaxTemp'), 
                     avg('Rainfall'), 
                     avg('WindGustSpeed'), 
                     avg('WindSpeed9am'), 
                     avg('WindSpeed3pm'), 
                     avg('Humidity9am'), 
                     avg('Humidity3pm'), 
                     avg('Pressure9am'), 
                     avg('Pressure3pm')).collect()

+------------------+------------------+------------------+------------------+-----------------+-----------------+----------------+------------------+------------------+-----------------+
|      avg(MinTemp)|      avg(MaxTemp)|     avg(Rainfall)|avg(WindGustSpeed)|avg(WindSpeed9am)|avg(WindSpeed3pm)|avg(Humidity9am)|  avg(Humidity3pm)|  avg(Pressure9am)| avg(Pressure3pm)|
+------------------+------------------+------------------+------------------+-----------------+-----------------+----------------+------------------+------------------+-----------------+
|12.186399728729098|23.226784191272444|2.3499740743111954| 39.98429165757619|  14.001988000994|18.63757586179718|68.8438103105705|51.482606091656265|1017.6537584159781|1015.258203537907|
+------------------+------------------+------------------+------------------+-----------------+-----------------+----------------+------------------+------------------+-----------------+



In [11]:
from pyspark.sql.functions import max

df.select(max('WindGustDir'),
          max('WindDir9am'),
          max('WindDir3pm'),
          max('RainToday'),
          max('RainTomorrow'),
         ).show()

max_list = df.select(max('WindGustDir'),
                     max('WindDir9am'),
                     max('WindDir3pm'),
                     max('RainToday'),
                     max('RainTomorrow'),
                    ).collect()

+----------------+---------------+---------------+--------------+-----------------+
|max(WindGustDir)|max(WindDir9am)|max(WindDir3pm)|max(RainToday)|max(RainTomorrow)|
+----------------+---------------+---------------+--------------+-----------------+
|             WSW|            WSW|            WSW|           Yes|              Yes|
+----------------+---------------+---------------+--------------+-----------------+



In [12]:
from pyspark.sql.functions import when

df = df\
.withColumn('MinTemp', when(df.MinTemp=='NA', avg_list[0][0]).otherwise(df.MinTemp))\
.withColumn('MaxTemp', when(df.MaxTemp=='NA', avg_list[0][1]).otherwise(df.MaxTemp))\
.withColumn('Rainfall', when(df.Rainfall=='NA', avg_list[0][2]).otherwise(df.Rainfall))\
.withColumn('WindGustSpeed', when(df.WindGustSpeed=='NA', avg_list[0][3]).otherwise(df.WindGustSpeed))\
.withColumn('WindSpeed9am', when(df.WindSpeed9am=='NA', avg_list[0][4]).otherwise(df.WindSpeed9am))\
.withColumn('WindSpeed3pm', when(df.WindSpeed3pm=='NA', avg_list[0][5]).otherwise(df.WindSpeed3pm))\
.withColumn('Humidity9am', when(df.Humidity9am=='NA', avg_list[0][6]).otherwise(df.Humidity9am))\
.withColumn('Humidity3pm', when(df.Humidity3pm=='NA', avg_list[0][7]).otherwise(df.Humidity3pm))\
.withColumn('Pressure9am', when(df.Pressure9am=='NA', avg_list[0][8]).otherwise(df.Pressure9am))\
.withColumn('Pressure3pm', when(df.Pressure3pm=='NA', avg_list[0][9]).otherwise(df.Pressure3pm))\
.withColumn('WindGustDir', when(df.WindGustDir=='NA', max_list[0][0]).otherwise(df.WindGustDir))\
.withColumn('WindDir9am', when(df.WindDir9am=='NA', max_list[0][1]).otherwise(df.WindDir9am))\
.withColumn('WindDir3pm', when(df.WindDir3pm=='NA', max_list[0][2]).otherwise(df.WindDir3pm))\
.withColumn('RainToday', when(df.RainToday=='NA', max_list[0][3]).otherwise(df.RainToday))\
.withColumn('RainTomorrow', when(df.RainTomorrow=='NA', max_list[0][4]).otherwise(df.RainTomorrow))\

df.show()

+-------+-------+------------------+-----------+-------------+----------+----------+---------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|MinTemp|MaxTemp|          Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|   WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|
+-------+-------+------------------+-----------+-------------+----------+----------+---------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|   13.4|   22.9|               0.6|          W|           44|         W|       WNW|             20|          24|         71|         22|     1007.7|     1007.1|       No|          No|
|    7.4|   25.1|                 0|        WNW|           44|       NNW|       WSW|              4|          22|         44|         25|     1010.6|     1007.8|       No|          No|
|   12.9|   25.7|                 0|        WSW|           46|         W|  

## Step 06: Data transformation

In [13]:
column_list = ['MinTemp', 
               'MaxTemp', 
               'Rainfall', 
               'WindGustSpeed', 
               'WindSpeed9am', 
               'WindSpeed3pm', 
               'Humidity9am', 
               'Humidity3pm', 
               'Pressure9am', 
               'Pressure3pm',
               'WindGustDir',
               'WindDir9am',
               'WindDir3pm',
               'RainToday',               
               'RainTomorrow'
              ]

for column in column_list:
    print(df.schema[column].dataType)

StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType
StringType


In [14]:
from pyspark.sql.types import DoubleType

df = df\
.withColumn('MinTemp', df['MinTemp'].cast(DoubleType()))\
.withColumn('MaxTemp', df['MaxTemp'].cast(DoubleType()))\
.withColumn('Rainfall', df['Rainfall'].cast(DoubleType()))\
.withColumn('WindGustSpeed', df['WindGustSpeed'].cast(DoubleType()))\
.withColumn('WindSpeed9am', df['WindSpeed9am'].cast(DoubleType()))\
.withColumn('WindSpeed3pm', df['WindSpeed3pm'].cast(DoubleType()))\
.withColumn('Humidity9am', df['Humidity9am'].cast(DoubleType()))\
.withColumn('Humidity3pm', df['Humidity3pm'].cast(DoubleType()))\
.withColumn('Pressure9am', df['Pressure9am'].cast(DoubleType()))\
.withColumn('Pressure3pm', df['Pressure3pm'].cast(DoubleType()))\

In [15]:
from pyspark.ml.feature import StringIndexer

df = StringIndexer(inputCol="WindGustDir", outputCol="WindGustDirIndex").fit(df).transform(df)
df = StringIndexer(inputCol="WindDir9am", outputCol="WindDir9amIndex").fit(df).transform(df)
df = StringIndexer(inputCol="WindDir3pm", outputCol="WindDir3pmIndex").fit(df).transform(df)
df = StringIndexer(inputCol="RainToday", outputCol="RainTodayIndex").fit(df).transform(df)
df = StringIndexer(inputCol="RainTomorrow", outputCol="RainTomorrowIndex").fit(df).transform(df)
df = df.withColumn('WindGustDir', df['WindGustDirIndex'])
df = df.withColumn('WindDir9am', df['WindDir9amIndex'])
df = df.withColumn('WindDir3pm', df['WindDir3pmIndex'])
df = df.withColumn('RainToday', df['RainTodayIndex'])
df = df.withColumn('RainTomorrow', df['RainTomorrowIndex'])
df.show()

+-------+-------+------------------+-----------+-------------+----------+----------+---------------+------------+-----------+-----------+-----------+-----------+---------+------------+----------------+---------------+---------------+--------------+-----------------+
|MinTemp|MaxTemp|          Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|   WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|WindGustDirIndex|WindDir9amIndex|WindDir3pmIndex|RainTodayIndex|RainTomorrowIndex|
+-------+-------+------------------+-----------+-------------+----------+----------+---------------+------------+-----------+-----------+-----------+-----------+---------+------------+----------------+---------------+---------------+--------------+-----------------+
|   13.4|   22.9|               0.6|        1.0|         44.0|       7.0|       7.0|           20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|      0.0|         0.0|             1.

In [16]:
for column in column_list:
    print(df.schema[column].dataType)

DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType
DoubleType


In [17]:
df = df.select(*column_list)
df.show(0)

+-------+-------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+-----------+----------+----------+---------+------------+
|MinTemp|MaxTemp|Rainfall|WindGustSpeed|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|WindGustDir|WindDir9am|WindDir3pm|RainToday|RainTomorrow|
+-------+-------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+-----------+----------+----------+---------+------------+
+-------+-------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+-----------+----------+----------+---------+------------+
only showing top 0 rows



## Step 07: Create the feature vector and divide the dataset

In [18]:
from pyspark.ml.feature import VectorAssembler

feature_list = ['MinTemp', 
               'MaxTemp', 
               'Rainfall', 
               'WindGustSpeed', 
               'WindSpeed9am', 
               'WindSpeed3pm', 
               'Humidity9am', 
               'Humidity3pm', 
               'Pressure9am', 
               'Pressure3pm',
               'WindGustDir',
               'WindDir9am',
               'WindDir3pm',
               'RainToday',               
              ]


va = VectorAssembler(inputCols=feature_list, outputCol='features')
feature_df = va.transform(df)
feature_df = feature_df.withColumn('label', feature_df['RainTomorrow'])
feature_df.show()

+-------+-------+------------------+-------------+---------------+------------+-----------+-----------+-----------+-----------+-----------+----------+----------+---------+------------+--------------------+-----+
|MinTemp|MaxTemp|          Rainfall|WindGustSpeed|   WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|WindGustDir|WindDir9am|WindDir3pm|RainToday|RainTomorrow|            features|label|
+-------+-------+------------------+-------------+---------------+------------+-----------+-----------+-----------+-----------+-----------+----------+----------+---------+------------+--------------------+-----+
|   13.4|   22.9|               0.6|         44.0|           20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|        1.0|       7.0|       7.0|      0.0|         0.0|[13.4,22.9,0.6,44...|  0.0|
|    7.4|   25.1|               0.0|         44.0|            4.0|        22.0|       44.0|       25.0|     1010.6|     1007.8|        9.0|      10.0|  

In [19]:
train, test = feature_df.randomSplit([0.7, 0.3])

## Step 08: Apply machine learning classification algorithms on the dataset and compare their accuracy. Plot the accuracy as bar graph.

### Training

In [20]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator

lr = LogisticRegression(featuresCol = 'features', labelCol = 'RainTomorrow', maxIter=10)

paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()
    
# tvs = TrainValidationSplit(estimator=lr, trainRatio=0.8) # 80% for training, 20% for validation.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)
model = tvs.fit(train)

### Predicting

In [21]:
model.transform(test)\
    .select("features", "label", "prediction")\
    .show()

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|[-8.5,0.6,1.0,31....|  0.0|       0.0|
|[-8.2,15.2,0.0,17...|  0.0|       0.0|
|[-8.0,2.1,2.34997...|  0.0|       0.0|
|[-8.0,15.1,0.2,39...|  0.0|       0.0|
|[-7.6,-1.4,5.8,31...|  0.0|       1.0|
|[-7.5,-0.7,0.0,35...|  0.0|       0.0|
|[-7.3,3.2,1.0,61....|  0.0|       1.0|
|[-7.2,-1.7,0.8,57...|  1.0|       1.0|
|[-7.0,0.1,0.8,19....|  0.0|       0.0|
|[-7.0,10.5,0.0,28...|  0.0|       0.0|
|[-6.9,11.4,0.0,37...|  0.0|       0.0|
|[-6.9,14.0,0.0,30...|  0.0|       0.0|
|[-6.8,13.8,0.0,39...|  0.0|       0.0|
|[-6.8,14.5,5.8,48...|  0.0|       0.0|
|[-6.7,2.9,0.8,33....|  0.0|       0.0|
|[-6.7,15.7,0.0,37...|  0.0|       0.0|
|[-6.6,1.1,0.0,41....|  0.0|       0.0|
|[-6.6,1.4,0.2,30....|  0.0|       0.0|
|[-6.5,-3.8,8.4,52...|  1.0|       1.0|
|[-6.5,-2.0,4.6,57...|  0.0|       1.0|
+--------------------+-----+----------+
only showing top 20 rows



### Evaluating

In [23]:
predictions = lrModel.transform(test)
predictions.select(feature_list).show(10)

NameError: name 'lrModel' is not defined

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction", 
    metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

print("Accuracy: %f" % (accuracy))
print("Test Error: %f" % (1.0 - accuracy))

NameError: name 'predictions' is not defined

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

% matplotlib inline

beta = np.sort(lrModel.coefficients)
plt.plot(beta)
plt.ylabel('Beta Coefficients')
plt.show()

## Step 09: Calculate the confusion matrix and find the precision, recall, and F1 score of each classification algorithm. Explain how the accuracy of the predication can be improved?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
class_temp = predictions.select("label").groupBy("label")\
                        .count().sort('count', ascending=False).toPandas()
class_temp = class_temp["label"].values.tolist()
class_names = map(str, class_temp)
# # # print(class_name)
print(class_names)

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = predictions.select("label")
y_true = y_true.toPandas()

y_pred = predictions.select("prediction")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred) # labels=class_names
cnf_matrix

In [ ]:
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Confusion matrix, without normalization')
plt.show()

In [ ]:
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True, title='Normalized confusion matrix')
plt.show()